In [1]:
from pathlib import Path
import csv
from collections import defaultdict
import os.path
from statistics import mean 
import pandas as pd
import altair as alt
import fiona
import geopandas as gpd
import gpdvega
import json
import datetime
from matplotlib import cm
from matplotlib.colors import to_hex
import shapely
from shapely.geometry import Polygon

colormap = cm.spring

In [2]:
today = datetime.date.today()
today

datetime.date(2019, 7, 2)

In [3]:
str(datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S"))

'2019-07-02T16:18:31'

In [4]:
def read_csv(filename):
    with open(filename, newline='') as File:  
        reader = csv.reader(File)
        header1 = next(reader)
        header2 = next(reader)
        results = []
        for row in reader:
            folder = Path(row[0].strip("'"))
            parts = folder.parts
            if len(parts) > 2 and parts[-1].lower().endswith('.tif'):
                results.append({"folder": str(folder.parent), "hits": int(row[1]), "bytes": int(row[2]), "cost": float(row[3])})
        return results

In [5]:
def read_json(filename):
    with open(filename, encoding='utf-8') as File:  
        reader = json.loads(File.read())
        reader_files = reader['Files']
        today = datetime.date.today()  
        results = []
        for i,v in reader_files.items():
            if str(i).endswith('.TIF') or str(i).endswith('.tif') or str(i).endswith('.tiff'):
                results.append({"folder": str(i), "hits": int(v[0]), "bytes": int(v[1]), 'date':today})
        return results
    

In [6]:
all_entries = read_json("week201925.json")

In [7]:
len(all_entries)

99

In [8]:
def product_name(folder):
    parts = Path(folder).parts
    if parts[0] == 'mangrove_cover':
        return parts[0]
    return os.path.join(*parts[:2])


def spatial_id(folder):
    parts = Path(folder).parts
    print(parts)
    print(parts[-1])
    if parts[-2] in ['NBAR', 'NBART', 'QA', 'SUPPLEMENTARY', 'LAMBERTIAN']:
        print("OK")
    if len(parts) > 2 and parts[0] == 'L2' and parts[1] == 'sentinel-2-nrt' and parts[-2] in ['NBAR', 'NBART', 'QA', 'SUPPLEMENTARY', 'LAMBERTIAN']:
        try:
            print(parts[-3].split("_")[-2][1:])
            return parts[-3].split("_")[-2][1:]
        except IndexError:
            print(folder)
    else:
        return '<none>'

def merge_pre(folder, dicts):
    today = datetime.date.today() 
    return {
        'hits': max(int(d['hits']) for d in dicts),
        'bytes': sum(int(d['bytes']) for d in dicts),
        #'cost': sum(float(d['cost']) for d in dicts),
        'spatial_id': spatial_id(folder),
        'product': product_name(folder),
        'folder': folder,
        'date': str(datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S"))
    }

def group(entry_list, key):
    lookup = defaultdict(list)
    
    for d in entry_list:
        lookup[d[key]].append(d)
        
    return lookup
    

def merge(dicts):
    return {
        'hits': sum(int(d['hits']) for d in dicts),
        'bytes': sum(int(d['bytes']) for d in dicts),
        #'cost': sum(float(d['cost']) for d in dicts),
    }

In [9]:
stage1 = [merge_pre(key, value) for key, value in group(all_entries, 'folder').items()]

('hltc', 'v2.0.0', 'composite', 'low-tide', 'lon_117', 'lat_-20', 'COMPOSITE_LOW_135_117.6_-20.59_20100101_20170101_PER_20_green.tif')
COMPOSITE_LOW_135_117.6_-20.59_20100101_20170101_PER_20_green.tif
('hltc', 'v2.0.0', 'composite', 'low-tide', 'lon_117', 'lat_-20', 'COMPOSITE_LOW_135_117.6_-20.59_20100101_20170101_PER_20_red.tif')
COMPOSITE_LOW_135_117.6_-20.59_20100101_20170101_PER_20_red.tif
('hltc', 'v2.0.0', 'composite', 'low-tide', 'lon_117', 'lat_-20', 'COMPOSITE_LOW_135_117.6_-20.59_20100101_20170101_PER_20_blue.tif')
COMPOSITE_LOW_135_117.6_-20.59_20100101_20170101_PER_20_blue.tif
('L2', 'sentinel-2-nrt', 'S2MSIARD', '2019-06-19', 'S2B_OPER_MSI_ARD_TL_EPAE_20190619T011513_A011925_T55HFB_N02.07', 'NBART', 'NBART_B04.TIF')
NBART_B04.TIF
OK
55HFB
('L2', 'sentinel-2-nrt', 'S2MSIARD', '2019-06-13', 'S2B_OPER_MSI_ARD_TL_EPAE_20190613T023435_A011840_T53LPG_N02.07', 'NBART', 'NBART_B02.TIF')
NBART_B02.TIF
OK
53LPG
('L2', 'sentinel-2-nrt', 'S2MSIARD', '2019-06-19', 'S2B_OPER_MSI_ARD_TL

In [10]:
stage1

[{'hits': 1907,
  'bytes': 1343757048,
  'spatial_id': '<none>',
  'product': 'hltc/v2.0.0',
  'folder': 'hltc/v2.0.0/composite/low-tide/lon_117/lat_-20/COMPOSITE_LOW_135_117.6_-20.59_20100101_20170101_PER_20_green.tif',
  'date': '2019-07-02T16:18:35'},
 {'hits': 1805,
  'bytes': 1300606007,
  'spatial_id': '<none>',
  'product': 'hltc/v2.0.0',
  'folder': 'hltc/v2.0.0/composite/low-tide/lon_117/lat_-20/COMPOSITE_LOW_135_117.6_-20.59_20100101_20170101_PER_20_red.tif',
  'date': '2019-07-02T16:18:35'},
 {'hits': 1805,
  'bytes': 1234688567,
  'spatial_id': '<none>',
  'product': 'hltc/v2.0.0',
  'folder': 'hltc/v2.0.0/composite/low-tide/lon_117/lat_-20/COMPOSITE_LOW_135_117.6_-20.59_20100101_20170101_PER_20_blue.tif',
  'date': '2019-07-02T16:18:35'},
 {'hits': 1386,
  'bytes': 1134821244,
  'spatial_id': '55HFB',
  'product': 'L2/sentinel-2-nrt',
  'folder': 'L2/sentinel-2-nrt/S2MSIARD/2019-06-19/S2B_OPER_MSI_ARD_TL_EPAE_20190619T011513_A011925_T55HFB_N02.07/NBART/NBART_B04.TIF',
  'd

In [11]:
sentinel2 = [d for d in stage1 if d['product'] == 'L2/sentinel-2-nrt']

In [12]:
sentinel2

[{'hits': 1386,
  'bytes': 1134821244,
  'spatial_id': '55HFB',
  'product': 'L2/sentinel-2-nrt',
  'folder': 'L2/sentinel-2-nrt/S2MSIARD/2019-06-19/S2B_OPER_MSI_ARD_TL_EPAE_20190619T011513_A011925_T55HFB_N02.07/NBART/NBART_B04.TIF',
  'date': '2019-07-02T16:18:35'},
 {'hits': 2762,
  'bytes': 1105459735,
  'spatial_id': '53LPG',
  'product': 'L2/sentinel-2-nrt',
  'folder': 'L2/sentinel-2-nrt/S2MSIARD/2019-06-13/S2B_OPER_MSI_ARD_TL_EPAE_20190613T023435_A011840_T53LPG_N02.07/NBART/NBART_B02.TIF',
  'date': '2019-07-02T16:18:35'},
 {'hits': 1386,
  'bytes': 1101930328,
  'spatial_id': '55HFB',
  'product': 'L2/sentinel-2-nrt',
  'folder': 'L2/sentinel-2-nrt/S2MSIARD/2019-06-19/S2B_OPER_MSI_ARD_TL_EPAE_20190619T011513_A011925_T55HFB_N02.07/NBART/NBART_B03.TIF',
  'date': '2019-07-02T16:18:35'},
 {'hits': 1387,
  'bytes': 1073232052,
  'spatial_id': '55HFB',
  'product': 'L2/sentinel-2-nrt',
  'folder': 'L2/sentinel-2-nrt/S2MSIARD/2019-06-19/S2B_OPER_MSI_ARD_TL_EPAE_20190619T011513_A01192

In [ ]:
s2_summary = {key: merge(value)['hits'] for key, value in group(sentinel2, 'spatial_id').items()}

In [ ]:
s2_summary

In [ ]:
s2_summary_list = list(s2_summary.items())

In [ ]:
item= []
sentinel2_list_a = (item.append(itera.items()) for itera in sentinel2)

In [ ]:
new_sentinel2 = 

In [ ]:
sentinel2_list_a

In [ ]:
s2_summary_list

In [ ]:
s2_mgrs = [entry[0] for entry in s2_summary_list]
s2_hits = [entry[1] for entry in s2_summary_list]

In [ ]:
hit_values = pd.DataFrame({'MGRS': s2_mgrs, 'hits': s2_hits})

In [13]:
with open('australian-mgrs-tiles.geojson') as fl:
    input_gj = json.load(fl)

In [14]:
feats = input_gj['features']

In [ ]:
for feat in feats:
    print(feat["geometry"]['coordinates'])

In [ ]:
for feat in feats:
    polygon = Polygon(feat["geometry"]['coordinates'][0])
    print(polygon.centroid)
    print(polygon.centroid.y)
    
    print(feat["geometry"]['coordinates'])
    #a , b = (numpy.average(feat["geometry"]['coordinates'][0][0],feat["geometry"]['coordinates'][0][2]),numpy.average(feat["geometry"]['coordinates'][0][1],feat["geometry"]['coordinates'][0][3]) )

In [ ]:
import numpy 
import shapely 

In [ ]:
from shapely.geometry import Polygon
polygon = Polygon([[107.4059053046, -41.4951070527, 0.0],[108.7199670857, -41.5286954845, 0.0] ,[108.6843689071, -42.5168269107, 0.0],[107.3498806889, -42.4820618042, 0.0] ])
print(polygon.centroid)
print(polygon.centroid.coords)
dir(polygon.centroid)

In [ ]:
for feat in feats:
    if 'MGRS' in feat['properties']:
        
  

In [ ]:
for feat in feats:
    print(feat['properties'])
    if 'MGRS' in feat['properties']:
        mgrs = feat['properties']['MGRS']
        if mgrs in s2_summary:
            feat['properties']['hits'] = s2_summary[mgrs]
            feat['properties']['stroke'] = "#ff00ff"
            feat['properties']['fill-opacity'] = 0.5
            feat['properties']['fill'] = "#ff0000"
            if feat['properties']['hits'] > 100:
                feat['properties']['fill'] = "#0000ff"
                

In [ ]:
grp = group(sentinel2, 'spatial_id')

In [ ]:
x = 2.34455555
y = round(x,1)
y

In [15]:
for feat in feats:
    if 'MGRS' in feat['properties']:
        mgrs = feat['properties']['MGRS']
        count=0
        for dict_item in sentinel2:
            if (dict_item['spatial_id']) == mgrs:
                polygon = Polygon(feat["geometry"]['coordinates'][0])
                dict_item['Lat'] = round(polygon.centroid.y, 1)
                dict_item['Lon'] = round(polygon.centroid.x, 1) 
                

In [16]:
sentinel2

[{'hits': 1386,
  'bytes': 1134821244,
  'spatial_id': '55HFB',
  'product': 'L2/sentinel-2-nrt',
  'folder': 'L2/sentinel-2-nrt/S2MSIARD/2019-06-19/S2B_OPER_MSI_ARD_TL_EPAE_20190619T011513_A011925_T55HFB_N02.07/NBART/NBART_B04.TIF',
  'date': '2019-07-02T16:18:35',
  'Lat': -34.8,
  'Lon': 148.7},
 {'hits': 2762,
  'bytes': 1105459735,
  'spatial_id': '53LPG',
  'product': 'L2/sentinel-2-nrt',
  'folder': 'L2/sentinel-2-nrt/S2MSIARD/2019-06-13/S2B_OPER_MSI_ARD_TL_EPAE_20190613T023435_A011840_T53LPG_N02.07/NBART/NBART_B02.TIF',
  'date': '2019-07-02T16:18:35',
  'Lat': -12.3,
  'Lon': 136.4},
 {'hits': 1386,
  'bytes': 1101930328,
  'spatial_id': '55HFB',
  'product': 'L2/sentinel-2-nrt',
  'folder': 'L2/sentinel-2-nrt/S2MSIARD/2019-06-19/S2B_OPER_MSI_ARD_TL_EPAE_20190619T011513_A011925_T55HFB_N02.07/NBART/NBART_B03.TIF',
  'date': '2019-07-02T16:18:35',
  'Lat': -34.8,
  'Lon': 148.7},
 {'hits': 1387,
  'bytes': 1073232052,
  'spatial_id': '55HFB',
  'product': 'L2/sentinel-2-nrt',
  

In [ ]:
!ls

In [17]:
file = open("data.csv", "w")

In [18]:
keys = sentinel2[0].keys()

In [19]:
keys

dict_keys(['hits', 'bytes', 'spatial_id', 'product', 'folder', 'date', 'Lat', 'Lon'])

In [20]:
with open('data.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(sentinel2)

In [21]:
cat data.csv

In [ ]:
for dict_item in sentinel2:
    print(dict_item)
    print(dict_item['hits'])
    print(dict_item['spatial_id'])


In [ ]:
with open('test.geojson', 'w') as fl:
    json.dump(input_gj, fl)

In [ ]:
input_gj

In [ ]:
type(feats)

In [ ]:
ex_feat = feats[0]

In [ ]:
list(ex_feat['properties'])

In [ ]:
with open('low_composite_20.geojson') as fl:
    low_comp = json.load(fl)

In [ ]:
list(low_comp['features'][0]['properties'])

In [ ]:
mgrs_poly = {}

In [ ]:
for entry in grid:
    mgrs = entry['properties']['MGRS']
    mgrs_poly[mgrs] = entry

In [ ]:
c234 = set(mgrs_poly)

In [ ]:
geojson2 = [mgrs_poly[x] for x in c234]

In [ ]:
common = set(s2_summary) & set(mgrs_poly)

In [ ]:
geojson = [mgrs_poly[x] for x in common]

In [ ]:
alt.Chart(alt.Data(values=geojson)).mark_geoshape(stroke='black', strokeWidth=1).encode().properties(width=400, height=400)

In [ ]:
sentinel2[0]['spatial_id']

In [ ]:
polygons = gpd.GeoDataFrame.from_features((geojson))

In [ ]:
polygons.head()

In [ ]:
alt.Chart(merged).mark_geoshape(
        stroke='black',
        strokeWidth=1
    ).encode(
    ).properties(
        width=400,
        height=400
    )

In [ ]:
merged = polygons.merge(hit_values, on= 'MGRS', how = 'inner')[['geometry', 'hits', 'MGRS']]

In [ ]:
merged.head()

In [ ]:
def gen_map(geodata, color_column, title):
    '''Generates DC ANC map with population choropleth and ANC labels'''
    # Add Base Layer
    base = alt.Chart(geodata, title = title).mark_geoshape(
        stroke='black',
        strokeWidth=1
    ).encode(
    ).properties(
        width=400,
        height=400
    )
    # Add Choropleth Layer
    choro = alt.Chart(geodata).mark_geoshape(
        fill='lightgray',
        stroke='black'
    ).encode(
        alt.Color(color_column, 
                  type='quantitative', 
                 scale=alt.Scale(scheme='bluegreen'),
                 title = "hits")
    )

    return base + choro


In [ ]:
gen_map(merged, 'hits', 'sentinel-2')

In [ ]:
gen_map(polygons, 'hits', 'sentinel-2')

In [ ]:

  
def gen_base(geojson):
    '''Generates baselayer of DC ANC map'''
    base = alt.Chart(alt.Data(values=geojson)).mark_geoshape(
        stroke='black',
        strokeWidth=1
    ).encode(
    ).properties(
        width=400,
        height=400
    )
    return base



In [ ]:
import requests

In [ ]:
geos = "australia-mgrs-grid.geojson"

In [ ]:
gen_base(grid)

In [ ]:
polygons.plot()

In [ ]:
geojson

In [ ]:
gdf_js = gpd.read_file("australia-mgrs-grid.geojson")

In [ ]:
gdf_js

In [ ]:
gdf_js.shape

In [ ]:
gdf_js.plot()

In [ ]:
%matplotlib

In [ ]:
gdf_js.GZD.unique()

In [ ]:
from matplotlib import cm
from matplotlib.colors import to_hex

In [ ]:
to_hex(colormap(1))

In [ ]:
to_hex(colormap(0))

In [ ]:
import datetime

In [ ]:
today=datetime.date.today()
yesterday =(today - datetime.timedelta(days =2)).strftime("%Y%m%d")

In [ ]:
today

In [ ]:
yesterday

In [ ]:
week = today.strftime("%Y%V")

In [ ]:
week

In [ ]:
month= today.strftime("%Y%m")

In [ ]:
month